In [ ]:
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import pdfo
import scipy.optimize as opt

In [ ]:
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

# Data, kovariance, vlastní hodnoty

In [ ]:
data = np.load("data/dataset/sel_voxels_from_4_ds.npy")
print(data.shape)

In [ ]:
cov_m = np.cov(data.T)

sn.heatmap(cov_m)
plt.show()

In [ ]:
print(np.min(cov_m))

In [ ]:
e_val, e_vec = np.linalg.eig(cov_m)

In [ ]:
plt.plot(e_val, marker='o')
plt.yscale("log")
plt.show()
plt.yscale("log")
plt.plot(e_val[:10], marker='o')
plt.show()

In [ ]:
cen = np.array([np.average(x) for x in data.T])

ti_seq = np.array([*list(range(50, 400, 25)),
          *list(range(400, 1000, 10)),
          1000, 1030, 1050, 1080, 1100, 1130, 1150, 1180, 1200, 1230, 1250, 1280, 1300, 1330, 1350, 1380,
          1400, 1450, 1500, 1550, 1600, 1650, 1700,
          1800, 1900, 2000, 2100, 2200, 2300, 2500, 3000])

plt.figure(figsize=(15, 10))
for i in range(6):
    plt.plot(ti_seq, e_vec[:,i], label="{}".format(i))
plt.legend()
plt.show()

In [ ]:
plt.plot(ti_seq, cen)
plt.show()

In [ ]:
test_data = data[79]
decentr = test_data - cen

vec_count = 7
coefs = []
for i in range(105):
    l = np.linalg.norm(e_vec[:,i])
    coefs.append(sum(decentr * e_vec[:,i]) / (l ** 2))

rec = np.copy(cen)
for i in range(vec_count):
    rec += coefs[i] * e_vec[:,i]

s = np.copy(cen)
for i in range(6):
    s += coefs[i] * e_vec[:,i]

plt.plot(ti_seq, test_data, label="data")
plt.plot(ti_seq, rec, label="7")
plt.plot(ti_seq, s, label="6")
plt.legend()
plt.show()

# Projekce teoretických křivek, různé varianty výpočtu chyby podle T1

In [ ]:
def model(TI,T1,M0=1.):
    return M0*np.abs(1.-2.*np.exp(-TI/T1))

def modcurve(T1,M0=1.):
    return model(ti_seq,T1,M0)

def curvecoord(curve,base):
    coord = np.dot(base.T,curve)
    bnorm = np.linalg.norm(base,axis=0)
    coord /= bnorm**2
    return coord

def curveproj(curve,base):
    coord = curvecoord(curve,base)
    proj = np.sum(base * coord,axis=1) 
    return coord,proj

def curveres(curve,base):
    _,proj = curveproj(curve,base)
    return np.linalg.norm(curve-proj)

In [ ]:
# kolik rozmeru podprostoru nas zajima

components = 6
base = e_vec[:,:components]

In [ ]:
# Pro kazde T1 z Trange a nekolik zafixovanych hodnot M0 generujeme krivky
# kazda se posune do stredu, udela jeji projekce do vybraneho pdprostoru,
# zmeri se vzdalenost krivky od projekce

T1range = range(400,1800,5)
M0range = [100,200,300,500,800,1000]

plt.figure(figsize=(12,8))
for M0 in M0range:
    res = []
    for T1 in T1range:
        t1curve=modcurve(T1,M0)
        t1cen = t1curve - cen
        res.append(curveres(t1cen,base))
    
    plt.plot(T1range,res,label=str(M0))
    
plt.legend()
plt.show()

In [ ]:
# Alternativne M0 spocitane tak, ze se stred nekolika krivek posune (hledanim spravneho M0) 
# co nejbliz stredu dat
models = []
T1start = np.array([580,650,710,810,1000,1250,1580])
for t1 in T1start:
    models.append(modcurve(t1,1200))
    
models = np.array(models).T
avgm = np.average(models,axis=1)

M0 = 1200 * curvecoord(cen,avgm)
print(M0)

plt.figure(figsize=(12,8))
plt.plot(T1range, list(map(lambda T1: curveres(modcurve(T1,M0),base),T1range)))
plt.show()

In [ ]:
# A nejdivoceji ... pro kazde T1 optimalizujeme zvlast M0 tak, aby vzdalenost od poprostoru
# byla co nejmensi

fig,ax1 = plt.subplots(figsize=(12,8))
 
T1range = list(range(400,3000,10))

mins = [
    opt.brent(lambda M0: curveres(modcurve(T1,M0)-cen,base),brack=(0,1000))
    for T1 in T1range
]

ax1.plot(T1range,list(map(lambda TM: curveres(modcurve(*TM)-cen,base),zip(T1range,mins))),color=colors[0])
ax1.set_ylabel('mindist',color=colors[0])
ax2 = ax1.twinx()
ax2.plot(T1range,np.array(mins),color=colors[1])
ax2.set_ylabel('M0',color=colors[1])
 
plt.show()

# Hodnoty T1 minimalizující chybu

In [ ]:
# vykouzlene casy (multigauss fit)
fitt1 = [ 533, 637, 787, 1008, 1245, 1616 ]

# Teoretické křivky vrstev, jejich projekce a báze pro skládání

In [ ]:
# projekce puvodniho pocatku souradneho systemu do lowdim

low0, high0 = curveproj(0-cen, base)

# kontrola, že je to ono (mají vyjít nuly)
base @ low0 - high0

In [ ]:
M0 = 800
 
# krivky techto casu, projekce do lowdim, a posun o puvodni pocatek
lowt1s = (np.array(
    list(map(
        lambda c: curvecoord(c-cen,base),
        map(lambda ti: modcurve(ti,M0),fitt1)
    ))
) - low0).T

# totéž, ale ve všech dimenzích
hight1s = (np.array(
    list(map(
        lambda c: curveproj(c-cen,base)[1],
        map(lambda ti: modcurve(ti,M0),fitt1)
    ))
) - high0).T

In [ ]:
# je to použitelná báze (10000 je furt OK)?
np.linalg.cond(lowt1s)

In [ ]:
# kontrola, mělo by být stejné
np.linalg.cond(hight1s)

In [ ]:
# křivky nejlépe odpovídající jednotlivým vrstvám

plt.figure(figsize=(12,8))

for i in range(6):
    plt.plot(ti_seq,hight1s[:,i],label=str(i))
    
plt.legend()
plt.show()

# Projekce voxelů, napočítání M0s

In [ ]:
sample = np.random.choice(data.shape[0],5)
sample

In [ ]:
# Takhle by to mělo být, ale vycházejí záporné koeficienty

plt.figure(figsize=(12,8))
for i,idx in enumerate(sample):
    lowc = curvecoord(data[idx]-cen,base)
    M0s = np.linalg.solve(lowt1s,lowc-low0)
    M0s = np.round(M0s,3)
    
    print(idx,M0s)
    highc = hight1s @ M0s
    plt.plot(ti_seq,highc,label=str(idx),color=colors[i])
    plt.plot(ti_seq,data[idx],marker='.',color=colors[i],ls='')
    
plt.legend()
plt.show()

In [ ]:
# Zakážeme záporné koeficienty a je to tam

plt.figure(figsize=(12,8))

for i,idx in enumerate(sample):
    lowc = curvecoord(data[idx]-cen,base)
    
    M0s = opt.lsq_linear(lowt1s,lowc-low0,bounds=(0,np.inf))['x']
    M0s = np.round(M0s,3)

    print(idx,M0s)
    highc = hight1s @ M0s
    plt.plot(ti_seq,highc,label=str(idx),color=colors[i])
    plt.plot(ti_seq,data[idx],marker='.',color=colors[i],ls='')
    
plt.legend()
plt.show()

In [ ]:
# 10 sousedních voxelů, mělo by vycházet rouzumně spojitě

plt.figure(figsize=(12,8))

idx = sample[0]
for i in range(10):
    lowc = curvecoord(data[idx+i]-cen,base)
     
    M0s = opt.lsq_linear(lowt1s,lowc-low0,bounds=(0,np.inf))['x']
    M0s = np.round(M0s,3)

    print(idx+i,M0s)
        
    highc = hight1s @ M0s
    plt.plot(ti_seq,highc,label=str(idx+i),color=colors[i])
    plt.plot(ti_seq,data[idx+i],marker='.',color=colors[i],ls='')
    
plt.legend()
plt.show()